In [1]:
import cv2
from tensorflow.keras.models import load_model
import numpy as np
import imutils
from imutils.contours import sort_contours
import matplotlib as plt


def find_contours(img):
  conts = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  conts = imutils.grab_contours(conts)
  conts = sort_contours(conts, method = 'left-to-right')[0]
  return conts

def extract_roi(img, margin=2):
  roi = img[y - margin :y + h, x-margin: x + w + margin]
  return roi

def thresholding(img):
  thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
  return thresh

def resize_img(img, w, h):
  if w > h:
    resized = imutils.resize(img, width = 28)
  else:
    resized = imutils.resize(img, height = 28)

  (h, w) = resized.shape
  dX = int(max(0, 28 - w) / 2.0)
  dY = int(max(0, 28 - h) / 2.0)

  filled = cv2.copyMakeBorder(resized, top=dY, bottom=dY, right=dX, left=dX, borderType=cv2.BORDER_CONSTANT, value = (0,0,0))
  filled = cv2.resize(filled, (28,28))
  return filled

def normalization(img):
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=-1)
    return img
  
def process_box(gray,x,y,w,h):
    roi= extract_roi(gray)
    thresh = thresholding(roi)
    (h,w)= thresh.shape
    resized = resize_img(thresh,w,h)
    # cv2.imshow(" ", resized)
    # cv2.waitKey(0)
    normalized= normalization(resized)
    characters.append((normalized, (x,y,w,h)))
    
characters = []


In [ ]:
def preprocess_image(img):
    gray= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    value, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
    img = cv2.resize(thresh,(28,28))
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=-1)
    img = np.reshape(img, (1,28,28,1))
    
    return img

In [ ]:
name_labels = '0123456789'
name_labels+="ABCDEFGHIJKLMNOPQRSTUVWXYZ"
name_labels = [l  for l in name_labels]

In [ ]:
model = load_model(r"C:\Users\Enes\Desktop\OCR\network")

In [ ]:
characters = []
boxes = []
img= cv2.imread("test5.jpeg")
img_copy2= img.copy()
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (3,3), 0)
adaptive = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 9)
invertion = 255 - adaptive
dilation = cv2.dilate(invertion, np.ones((3,3)))

# cv2.imshow("gorsel",dilation)
# cv2.waitKey(0)

In [ ]:
#edges = cv2.Canny(dilation, 40, 150)
#dilation = cv2.dilate(edges, np.ones((3,3)))

In [ ]:
conts = find_contours(dilation.copy())

In [ ]:
min_w, max_w = 4, 160
min_h, max_h = 14, 140
img_copy = img.copy()
for c in conts:
  (x, y, w, h) = cv2.boundingRect(c)
  if (w >= min_w and w <= max_w) and (h >= min_h and h <= max_h):
    roi = gray[y:y+h, x:x+w]
    thresh = cv2.threshold(roi, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    cv2.rectangle(img_copy, (x, y), (x + w, y + h), (255, 100, 0), 2)

In [ ]:
(x, y, w, h) = cv2.boundingRect(conts[6])
print(x,y,w,h)
test_img = thresholding(gray[y:y+h, x:x+w])

(h,w)= test_img.shape
print(w,h)
test_img2= resize_img(test_img,w,h)
# cv2.imshow("görsel",test_img2)
# cv2.waitKey(0)
print(test_img2.shape)

In [ ]:
test_img2.shape, normalization(test_img2).shape

In [ ]:
for c in conts:
    (x,y,w,h)= cv2.boundingRect(c)
    if (w >= min_w and w <= max_w) and (h >= min_h and h <= max_h):
        process_box(gray ,x,y,w,h)

In [ ]:
characters[1]

In [ ]:
boxes = [box[1] for box in characters]
boxes

In [ ]:
pixels = np.array([ pixel[0] for pixel in characters], dtype='float32')
pixels.shape

In [ ]:
digits= "0123456789"
letters = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
character_list= digits+letters
character_list = [l for l in character_list]

In [ ]:
predictions= model.predict(pixels)
predictions.shape

In [ ]:
img_copy= img_copy2
results = []
for (prediction, (x,y,w,h)) in zip(predictions, boxes):
    i = np.argmax(prediction)
    #print(i)
    probability = prediction[i]
    character = character_list[i]
    cv2.rectangle(img_copy, (x,y) ,(x+w, y+h),(255,100,0),2)
    cv2.putText(img_copy,character, (x-5, y-10), cv2.FONT_HERSHEY_SIMPLEX,1.1, (0,0,0), 2)
    print(character,"->",probability)
    #cv2.imshow("detected letters",img_copy)
    #cv2.waitKey(0)
    results.append(character)
deger = "".join(results)
print(deger)
# cv2.imshow("detected letters",img_copy)
# cv2.waitKey(0)
cv2.imwrite("output345.png",img_copy)
results = []
    

In [ ]:
import cv2
from tensorflow.keras.models import load_model
import numpy as np
import imutils
from imutils.contours import sort_contours

def find_contours(img):
    conts = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    conts = imutils.grab_contours(conts)
    conts = sort_contours(conts, method='left-to-right')[0]
    return conts

def extract_roi(img, x, y, w, h, margin=2):
    roi = img[y - margin:y + h + margin, x - margin:x + w + margin]
    return roi

def thresholding(img):
    thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    return thresh

def resize_img(img, w, h):
    if w > h:
        resized = imutils.resize(img, width=28)
    else:
        resized = imutils.resize(img, height=28)

    (h, w) = resized.shape
    dX = int(max(0, 28 - w) / 2.0)
    dY = int(max(0, 28 - h) / 2.0)

    filled = cv2.copyMakeBorder(resized, top=dY, bottom=dY, right=dX, left=dX, borderType=cv2.BORDER_CONSTANT, value=(0, 0, 0))
    filled = cv2.resize(filled, (28, 28))
    return filled

def normalization(img):
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=-1)
    return img

def process_box(gray, x, y, w, h):
    roi = extract_roi(gray, x, y, w, h)
    thresh = thresholding(roi)
    (h, w) = thresh.shape
    resized = resize_img(thresh, w, h)
    normalized = normalization(resized)
    characters.append((normalized, (x, y, w, h)))

def preprocess_image(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    value, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
    img = cv2.resize(thresh, (28, 28))
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=-1)
    img = np.reshape(img, (1, 28, 28, 1))
    return img

# Load model
model = load_model(r"C:\Users\Enes\Desktop\OCR\network")

# Character list
character_list = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ'
character_list = [l for l in character_list]





In [ ]:
# Initialize lists
characters = []
boxes = []
# Read image
img = cv2.imread("handwrittenTest.jpeg")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (3, 3), 0)
adaptive = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 9)
invertion = 255 - adaptive
dilation = cv2.dilate(invertion, np.ones((3, 3)))

# Find contours
conts = find_contours(dilation.copy())
min_w, max_w = 4, 160
min_h, max_h = 14, 140

# Process each contour
for c in conts:
    (x, y, w, h) = cv2.boundingRect(c)
    if (w >= min_w and w <= max_w) and (h >= min_h and h <= max_h):
        process_box(gray, x, y, w, h)

# Extract boxes and pixels
boxes = [box[1] for box in characters]
pixels = np.array([pixel[0] for pixel in characters], dtype='float32')

# Make predictions
predictions = model.predict(pixels)
img_copy = img.copy()

# Function to group characters into words
def group_characters_into_words(boxes):
    if not boxes:
        return []

    # Calculate average width of characters and spaces between them
    average_width = np.mean([w for _, _, w, _ in boxes])
    spaces = [boxes[i + 1][0] - (boxes[i][0] + boxes[i][2]) for i in range(len(boxes) - 1)]
    avg_space = np.mean(spaces)
    threshold = avg_space * 6

    words = []
    current_word = []
    last_x = None

    for (x, y, w, h) in boxes:
        if last_x is not None and (x - last_x) > threshold:
            words.append(current_word)
            current_word = []
        current_word.append((x, y, w, h))
        last_x = x + w

    if current_word:
        words.append(current_word)

    return words

# Group characters into words
words = group_characters_into_words(boxes)

results = []
for word in words:
    word_text = ""
    for (x, y, w, h) in word:
        idx = boxes.index((x, y, w, h))
        prediction = predictions[idx]
        i = np.argmax(prediction)
        probability = prediction[i]
        character = character_list[i]
        cv2.rectangle(img_copy, (x, y), (x + w, y + h), (255, 100, 0), 2)
        cv2.putText(img_copy, character, (x - 5, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.1, (0, 0, 0), 2)
        word_text += character
    results.append(word_text)

#deger = " ".join(results)
print(results)

# cv2.imshow("detected letters", img_copy)
# cv2.waitKey(0)
cv2.imwrite("output345.png", img_copy)


In [38]:
def find_contours(img):
    conts = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    conts = imutils.grab_contours(conts)
    conts = sort_contours(conts, method='left-to-right')[0]
    return conts

def extract_roi(img, x, y, w, h, margin=2):
    roi = img[y - margin:y + h + margin, x - margin:x + w + margin]
    return roi

def thresholding(img):
    thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    return thresh

def resize_img(img, w, h):
    if w > h:
        resized = imutils.resize(img, width=28)
    else:
        resized = imutils.resize(img, height=28)

    (h, w) = resized.shape
    dX = int(max(0, 28 - w) / 2.0)
    dY = int(max(0, 28 - h) / 2.0)

    filled = cv2.copyMakeBorder(resized, top=dY, bottom=dY, right=dX, left=dX, borderType=cv2.BORDER_CONSTANT, value=(0, 0, 0))
    filled = cv2.resize(filled, (28, 28))
    return filled

def normalization(img):
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=-1)
    return img

def process_box(gray, x, y, w, h):
    roi = extract_roi(gray, x, y, w, h)
    thresh = thresholding(roi)
    (h, w) = thresh.shape
    resized = resize_img(thresh, w, h)
    normalized = normalization(resized)
    characters.append((normalized, (x, y, w, h)))

def preprocess_image(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    value, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
    img = cv2.resize(thresh, (28, 28))
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=-1)
    img = np.reshape(img, (1, 28, 28, 1))
    return img

characters = []
boxes = []
def group_characters_into_words(boxes):
    if not boxes:
        return []

    # Calculate the average width of characters and spaces between them
    widths = [w for _, _, w, _ in boxes]
    spaces = [boxes[i + 1][0] - (boxes[i][0] + boxes[i][2]) for i in range(len(boxes) - 1)]
    
    if not spaces:
        return [boxes]  # If there's only one box, it's one word

    avg_space = np.mean(spaces)
    threshold = avg_space * 3  # Adjust the multiplier as necessary

    words = []
    current_word = [boxes[0]]
    last_x = boxes[0][0] + boxes[0][2]

    for i in range(1, len(boxes)):
        x, y, w, h = boxes[i]
        if (x - last_x) > threshold:
            words.append(current_word)
            current_word = []
        current_word.append((x, y, w, h))
        last_x = x + w

    if current_word:
        words.append(current_word)

    return words

def predictImage(img):
    global characters, boxes
    characters = []
    boxes = []
    # Initialize lists

    model = load_model(r"C:\Users\Enes\Desktop\OCR\network")

    # Character list
    character_list = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    character_list = [l for l in character_list]
    # Initialize lists

    img = np.array(img)
    print("image type is",type(img))

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (3,3), 0)
    adaptive = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 9)
    invertion = 255 - adaptive
    dilation = cv2.dilate(invertion, np.ones((3,3)))

    # Find contours
    conts = find_contours(dilation.copy())
    min_w, max_w = 4, 160
    min_h, max_h = 14, 140
    
    # Process each contour
    for c in conts:
        (x, y, w, h) = cv2.boundingRect(c)
        if (w >= min_w and w <= max_w) and (h >= min_h and h <= max_h):
            process_box(gray, x, y, w, h)

    # Extract boxes and pixels
    boxes = [box[1] for box in characters]
    pixels = np.array([pixel[0] for pixel in characters], dtype='float32')
    
    # Make predictions
    predictions = model.predict(pixels)

    # Group characters into words
    words = group_characters_into_words(boxes)

    results = []
    for word in words:
        word_text = ""
        for (x, y, w, h) in word:
            idx = boxes.index((x, y, w, h))
            prediction = predictions[idx]
            i = np.argmax(prediction)
            probability = prediction[i]
            character = character_list[i]
            word_text += character
        results.append(word_text)
    return results

image = cv2.imread("test6buyuk.jpeg")

print(predictImage(image))

image type is <class 'numpy.ndarray'>
['ENUE5', 'A7', 'EL', '1AZJ1']
